# LISFLOOD calibrated parameters
***

**Author:** Chus Casado Rodríguez<br>
**Date:** 04-10-2023<br>

**Introduction:**<br>
This notebook extracts the LISFLOOD reservoir parameters calibrated in GloFAS for each catchment. The parameters are read from the XML settings file of every catchment.

Then, 

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path

In [2]:
from utils import xml_parameters

## Configuration

In [12]:
# location of the LISFLOOD models
path_glofas = Path('Z:/nahaUsers/casadje/GloFASv4/')
# name of the settings file
settings_file = 'settings_lisflood_wusewregion_FULLinit_GloFASnext-RunX.xml'
# name of the reservoirs static map
reservoir_map = '20210920_res_Global_03min.nc'

## Data
### Calibrated parameters

In [5]:
# catchment models available
catchments = [x for x in os.listdir(path_glofas / 'catchments') if os.path.isdir(os.path.join(path_glofas / f'catchments/{x}'))]

print(f'{len(catchments)} catchment models were found')

32 catchment models were found


In [6]:
# read calibrated parameters
parameters = {}
for catchment in catchments:
    xml = path_glofas / 'catchments' / catchment / 'settings' / settings_file
    parameters[catchment] = xml_parameters(xml, pars=['adjust_Normal_Flood', 'ReservoirRnormqMult'])
parameters = pd.DataFrame(parameters)

parameters

,453,476,477,527,555,556,642,656,662,695,...,825,829,830,831,835,836,854,856,858,865
adjust_Normal_Flood,0.139326,0.522563,0.638480,0.129179,0.501281,0.894556,0.01000,0.026621,0.480791,0.535129,...,0.325858,0.609061,0.958427,0.223297,0.544355,0.617579,0.387440,0.185101,0.982214,0.474467
ReservoirRnormqMult,0.250000,1.999610,0.828612,1.998290,0.946985,1.714180,1.59126,1.999900,1.256110,0.385927,...,1.843080,0.385225,0.453351,0.987671,0.372642,1.205480,0.705504,2.000000,0.451097,0.954613


In [19]:
# identify reservoirs in the catchment
reservoirs = {}
for catchment in catchments:
    try:
        # read NetCDF file
        nc = path_glofas / 'catchments' / catchment / 'maps' / reservoir_map
        res = xr.open_dataset(nc)['res'].data
        # extract ResID values
        res_list = res[~np.isnan(res)].astype(int).tolist()
        if len(res_list) > 0:
            reservoirs[catchment] = res_list
    except:
        print(f'The map {reservoir_map} could not be read in catchment {catchment}')
        
print(f'no. catchments: {len(reservoirs)}')
ids = np.unique([id for key, ls in reservoirs.items() for id in ls])
print(f'no. reservoirs: {len(ids)}')

no. catchments: 32
no. reservoirs: 46


In [17]:
reservoirs

{'453': [31, 169],
 '476': [609],
 '477': [169, 364],
 '527': [617],
 '555': [48],
 '556': [227, 14],
 '642': [146],
 '656': [621, 488, 54],
 '662': [193, 195],
 '695': [228],
 '741': [92, 12],
 '748': [7],
 '750': [92],
 '755': [50],
 '772': [362, 149],
 '788': [76, 56],
 '789': [386],
 '800': [446, 334],
 '807': [392, 140, 278],
 '809': [308, 334],
 '811': [56, 313],
 '820': [143, 17],
 '825': [446, 462, 258],
 '829': [313, 406],
 '830': [491],
 '831': [392, 140, 16, 174],
 '835': [362, 149, 446, 308, 334, 462, 258, 313, 359, 406, 449],
 '836': [359],
 '854': [442],
 '856': [315],
 '858': [393, 64],
 '865': [156]}